In [1]:
import io
import os
import pyarrow as pa
import pyarrow.fs
import requests
import urllib.request

In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 49766334464 (46.35 GB)
Present Capacity: 16162221180 (15.05 GB)
DFS Remaining: 15633235968 (14.56 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 3.27%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.20.0.3:9866 (p4_wadadkar-dn-1.p4_wadadkar_default)
Hostname: 0190684eeb36
Decommission Status : Normal
Configured Capacity: 24883167232 (23.17 GB)
DFS Used: 261765966 (249.64 MB)
Non DFS Used: 16788006066 (15.64 GB)
DFS Remaining: 7816617984 (7.28 GB)
DFS Used%: 1.05%
DFS Remaining%: 31.41%
Co

In [3]:
# Check if the file already exists and download it if not
import os
import urllib.request

file_path = "/nb/hdma-wi-2021.csv"
url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"

if not os.path.exists(file_path):
    print("File not found locally. Downloading...")
    urllib.request.urlretrieve(url, file_path)
    print("Download complete.")
else:
    print("File already exists. Skipping download.")

File already exists. Skipping download.


In [4]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

Deleted hdfs://boss:9000/single.csv
Deleted hdfs://boss:9000/double.csv


In [5]:
# Set the HDFS block size to 1MB and upload the file to two different locations with different replication factors
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -put /nb/hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -put /nb/hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [6]:
#q2
# Check the logical and physical sizes of the CSV files in HDFS
!hdfs dfs -du -h hdfs://boss:9000/single.csv
!hdfs dfs -du -h hdfs://boss:9000/double.csv

166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [7]:
#q3
file_url = "http://boss:9870/webhdfs/v1/single.csv"
params = {"op": "GETFILESTATUS"}
response=requests.get(file_url, params=params)
response.json()

{'FileStatus': {'accessTime': 1731652859388,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16388,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1731652863818,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [8]:
#q4
# Define the WebHDFS URL with noredirect=true
file_url = "http://boss:9870/webhdfs/v1/single.csv"  # Changed from single.parquet to single.csv
params = {
    "op": "OPEN",
    "offset": 0,
    "noredirect": "true"
}

# Send GET request
response = requests.get(file_url, params=params)
location = response.json().get("Location")
location

'http://0190684eeb36:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [9]:
#q5

# Define the WebHDFS URL and parameters for the OPEN operation
file_url = "http://boss:9870/webhdfs/v1/single.csv"
params = {
    "op": "GETFILESTATUS"
}
# Step 1: Get file size and block size
file_status = requests.get(file_url, params=params).json()
file_size = file_status["FileStatus"]["length"]
block_size = file_status["FileStatus"]["blockSize"]
num_blocks = (file_size + block_size - 1) // block_size  # Calculate total number of blocks

# Dictionary to store the block distribution by DataNode container ID
block_distribution = {}
# Step 2: Loop through each block to check its DataNode location
for block_num in range(num_blocks):
    # Set the offset for each block
    params = {
        "op": "OPEN",
        "offset": block_num * block_size,
        "noredirect": "true"
    }
    
    # Request block location
    response = requests.get(file_url, params=params)
    location = response.json().get("Location")
    
    # If location is found, extract the container ID
    if location:
        container_id = location.split("//")[1].split(":")[0]
    
    if container_id in block_distribution:
        block_distribution[container_id] += 1
    else:
        block_distribution[container_id] = 1
block_distribution

{'0190684eeb36': 78, 'd529d9fd4a2d': 89}

In [10]:
#q6
# Connect to HDFS using PyArrow
hdfs = pa.fs.HadoopFileSystem("boss", 9000)  # Replace "boss" with your Namenode hostname and "9000" with the correct port

# Define the path to the file on HDFS
file_path = "/single.csv"

# Open the file and read the first 10 bytes
with hdfs.open_input_file(file_path) as file:
    # Use read_at to read 10 bytes starting at offset 0
    first_10_bytes = file.read_at(10, 0)

first_10_bytes

2024-11-15 06:41:24,752 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [11]:
#q7

# Connect to HDFS on the Namenode at port 9000
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

# Open the file single.csv from HDFS
file_path = "/single.csv"
with hdfs.open_input_file(file_path) as file:
    # Wrap the NativeFile object in BufferedReader and TextIOWrapper for line-by-line reading
    buffer = io.BufferedReader(file)
    text_file = io.TextIOWrapper(buffer)

    # Count lines containing "Single Family"
    count = sum(1 for line in text_file if "Single Family" in line)
count

444874